In [1]:
import open3d as o3d
import copy
import numpy as np
import pickle 
import matplotlib.pyplot as plt
import os
from transforms import quaternion_to_rotation_matrix, transform_from_rot_and_pos, xyzw_to_rotation_matrix
import bayes3d as b
import jax.numpy as jnp

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
b.setup_visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
folder = 'panda_scans_v7'
data_list = []
for scan in os.listdir(folder):

    with open(folder+'/'+scan, 'rb') as pickle_file:
        data_list.append(pickle.load(pickle_file))


data = data_list[0] # 0 is cheezit, 1 is LEGO

colors = []
depths = []
poses = []

#for i in range(len(data)):



for i in range(8): # frame 8 of lego and after of lego have calibration issues
    colors.append(data[i]['camera_image']['rgbPixels'])
    depths.append(data[i]['camera_image']['depthPixels'])
    translation, quat = data[i]['camera_image']['camera_pose'] #quaternion_to_rotation_matrix, xyzw_to_rotation_matrix
    poses.append(transform_from_rot_and_pos(xyzw_to_rotation_matrix(np.array(quat)), np.array(translation)))


In [5]:
intrinsics= np.zeros((4,4))
intrinsics[:3,:3] = data[0]['camera_image']['camera_matrix'][0]
intrinsics[3,3] = 1
intrinsics[1,1] *= 1 #-1
#intrinsics = np.linalg.inv(intrinsics)
intrinsics = np.array(intrinsics)
intrinsics = b.Intrinsics(depths[0].shape[1], depths[0].shape[0],intrinsics[0,0],intrinsics[1,1],intrinsics[0,2],intrinsics[1,2],0,20)

HIINTERFACE = None
def carvekit_get_foreground_mask(image: b.RGBD):
    global HIINTERFACE
    if HIINTERFACE is None:
        from carvekit.api.high import HiInterface
        import torch
        HIINTERFACE = HiInterface(
            object_type="object",  # Can be "object" or "hairs-like".
            batch_size_seg=5,
            batch_size_matting=1,
            device='cuda' if torch.cuda.is_available() else 'cpu',
            seg_mask_size=640,  # Use 640 for Tracer B7 and 320 for U2Net
            matting_mask_size=2048,
            trimap_prob_threshold=220,#231,
            trimap_dilation=15,
            trimap_erosion_iters=20,
            fp16=False
        )
    imgs = HIINTERFACE([b.get_rgb_image(image.rgb)])
    mask = jnp.array(imgs[0])[...,-1] > 0.5
    return mask




In [6]:
pc = b.t3d.unproject_depth(depths[0], intrinsics)

b.clear()
b.show_cloud('1',pc.reshape(-1,3))

In [7]:


pcs = []
pcs_color = []

pcs_carved = []
pcs_color_carved = []

b.clear()

for i in range(len(colors)):
    pc = b.t3d.unproject_depth(depths[i], intrinsics)

    # plane_pose, plane_dims = b.utils.find_plane_and_dims(pc.reshape(-1,3), 
    # ransac_threshold=0.001, inlier_threshold=0.001, segmentation_threshold=1) #0.1)

    # if plane_pose[:3,2][2] > 0:
    #     plane_pose = plane_pose @ b.t3d.transform_from_axis_angle(jnp.array([1.0, 0.0, 0.0]), jnp.pi)

    # pc = b.t3d.apply_transform(pc, b.t3d.inverse_pose(plane_pose))
    


    indices = carvekit_get_foreground_mask(b.RGBD(colors[i], depths[i],np.linalg.inv(poses[i]),intrinsics))
    pcs_carved.append(pc[indices])
    pcs_color_carved.append(colors[i][indices]/255.0)

    pcs.append(pc.reshape(-1,3))
    pcs_color.append(colors[i].reshape(-1,3)/255.0)

In [8]:
pcs_reoriented = []
pcs_reoriented_carved = []

for i in range(len(pcs)):
    #b.show_cloud(str(i),b.t3d.apply_transform(pcs[i], np.linalg.inv(poses[i]) @ poses[0]) *10)
    pc_reoriented = b.t3d.apply_transform(pcs[i], poses[i])
    pcs_reoriented.append(pc_reoriented)
    pc_reoriented_carved = b.t3d.apply_transform(pcs_carved[i], poses[i])
    pcs_reoriented_carved.append(pc_reoriented_carved)
    b.show_cloud(str(i), pc_reoriented_carved*10)

In [9]:
## merge cloud and clean up with open3d
merged = np.concatenate(pcs_reoriented)
merged_carved = np.concatenate(pcs_reoriented_carved)

In [10]:
b.clear()

In [11]:
rand = np.random.RandomState(1234)

random_subsample = rand.choice(len(merged),20000)

plane_pose, plane_dims = b.utils.find_plane_and_dims(merged[random_subsample,:].reshape(-1,3), 
ransac_threshold=0.001, inlier_threshold=0.001, segmentation_threshold=0.1) #0.1)

if plane_pose[:3,2][2] > 0:
    print('flip')
    plane_pose = plane_pose @ b.t3d.transform_from_axis_angle(jnp.array([1.0, 0.0, 0.0]), jnp.pi)

In [12]:
b.show_pose('2',plane_pose)
b.show_cloud('1',merged[random_subsample,:])

In [13]:
b.clear()


    
merged_carved_reoriented = b.t3d.apply_transform(merged_carved, b.t3d.inverse_pose(plane_pose @ b.t3d.transform_from_axis_angle(jnp.array([1.0, 0.0, 0.0]), jnp.pi)))


In [14]:
b.show_cloud('1',merged_carved_reoriented)

In [16]:
### clean up points near plane
merged_carved_reoriented_threshold = merged_carved_reoriented[merged_carved_reoriented[:,2]>0.01]

# quck and dirty centering, preserves z axis
merged_carved_reoriented_threshold = np.array(merged_carved_reoriented_threshold)
merged_carved_reoriented_threshold[:,0] -= b.utils.aabb(merged_carved_reoriented_threshold)[1][0,4]
merged_carved_reoriented_threshold[:,1] -= b.utils.aabb(merged_carved_reoriented_threshold)[1][1,4]


### clean up an artifact
merged_carved_reoriented_threshold = merged_carved_reoriented_threshold[merged_carved_reoriented_threshold[:,1]>0.015]

In [17]:
b.show_cloud('1',merged_carved_reoriented_threshold)

In [18]:
mesh_threshold = 0.002
merged_carved_reoriented_threshold_voxelized = b.utils.voxelize(merged_carved_reoriented_threshold, mesh_threshold)
b.show_cloud('1',merged_carved_reoriented_threshold_voxelized)

In [21]:
segmented = b.utils.segment_point_cloud(merged_carved_reoriented_threshold_voxelized,threshold=0.02, min_points_in_cluster=5)
b.show_cloud('1',merged_carved_reoriented_threshold_voxelized[segmented == 0,:])

In [22]:
b.show_cloud('2',merged_carved_reoriented_threshold_voxelized[segmented == 1,:])

In [23]:
merged = jnp.concatenate([merged_carved_reoriented_threshold_voxelized[segmented == 0,:],merged_carved_reoriented_threshold_voxelized[segmented == 1,:]])
merged -= np.mean(merged,axis=0)

In [24]:
b.clear()
b.show_cloud('1',merged)

In [25]:
np.save('plane_pc_video_capture_hi_res', merged)

In [26]:
# merged_carved_reoriented_threshold = np.array(merged_carved_reoriented_threshold)
# merged_carved_reoriented_threshold[:,0] -= b.utils.aabb(merged_carved_reoriented_threshold)[1][0,4]
# merged_carved_reoriented_threshold[:,1] -= b.utils.aabb(merged_carved_reoriented_threshold)[1][1,4]
# b.show_cloud('1',merged_carved_reoriented_threshold)

In [27]:
mesh = b.utils.make_voxel_mesh_from_point_cloud(merged, mesh_threshold)
mesh = b.utils.center_mesh(mesh)

In [28]:

b.show_trimesh('1',mesh)

In [29]:
## write out files
name = 'toy_plane_video_capture_hi_res'
mesh.export(name+'.obj')
mesh.export(name+'.ply')
print('write mesh files: ' + name)

write mesh files: toy_plane_video_capture_hi_res


#### Open3D Remove Radius Outlier Carving

In [ ]:
random_subsample = np.random.choice(len(merged),20000)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(merged[random_subsample,:])

In [ ]:
b.show_cloud('1',np.asarray(pcd.points))

In [ ]:
scanned_pc = np.asarray(pcd.remove_radius_outlier(nb_points=200, radius=0.1)[0].points) # num = 50, rad = 0.025 for cheezit / nb_points=200, radius=0.015 for lego
scanned_pc = b.t3d.apply_transform(scanned_pc, b.t3d.inverse_pose(b.utils.aabb(scanned_pc)[1]))

b.show_cloud('1',scanned_pc*10)

In [ ]:
mesh = b.utils.make_voxel_mesh_from_point_cloud(scanned_pc, 0.01) # is this 2.5mm?, 0.0025 for lego
b.clear()
b.show_trimesh('1',mesh)